In [3]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

# api url
base_url = 'http://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo?serviceKey=9oUviJPMmodfrNLBPYD0czH%2BOoPnArWcnHsw6T5g2TwnKEs1dwXCp%2FI%2BNfeEGngIAK74J550ZjxJ0CxJy4cK7Q%3D%3D&resultType=json'

# rsi 함수
def calc_rsi(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        contents = response.text  # JSON 데이터 가져오기
    else:
        print(f"API 호출 실패: {response.status_code}")
    
    json_ob = json.loads(contents)

    data = json_ob['response']['body']['items']['item']
    selected_data = []

    for item in data:
        selected_item = {
            'basDt': item['basDt'],
            'itmsNm': item['itmsNm'],
            'mrktCtg': item['mrktCtg'],
            'clpr': item['clpr'],
            'vs': item['vs'],
            'fltRt': item['fltRt'],
            'mkp': item['mkp'],
            'hipr': item['hipr'],
            'lopr': item['lopr']
        }
        selected_data.append(selected_item)
        
    dataframe = pd.DataFrame(selected_data)

    # 데이터 프레임 열 이름 변경
    dataframe = dataframe.rename(columns={
        'basDt': '날짜',
        'itmsNm': '종목명',
        'mrktCtg': '시장구분',
        'clpr': '종가',
        'vs': '거래량',
        'fltRt': '등락률',
        'mkp': '시가총액',
        'hipr': '고가',
        'lopr': '저가'
    })
    
    # 날짜를 날짜 형식으로 변환
    dataframe['날짜'] = pd.to_datetime(dataframe['날짜'], format='%Y%m%d')

    # 종가를 수치로 변환
    dataframe['종가'] = pd.to_numeric(dataframe['종가'])

    # 날짜를 인덱스로 설정
    dataframe.set_index('날짜', inplace=True)
    
    # rsi 계산 함수
    period = 14
    # 가격의 차이 계산
    dataframe['Price Change'] = dataframe['종가'].diff()
    # 상승 가격과 하락 가격 구분
    dataframe['Gain'] = dataframe['Price Change'].apply(lambda x: x if x > 0 else 0)
    dataframe['Loss'] = dataframe['Price Change'].apply(lambda x: -x if x < 0 else 0)
    # 상승과 하락 가격의 이동 평균 계산
    dataframe['Avg Gain'] = dataframe['Gain'].rolling(window=period).mean()
    dataframe['Avg Loss'] = dataframe['Loss'].rolling(window=period).mean()  
    
    # Avoid division by zero by adding a small constant
    #dataframe['Avg Loss'] = dataframe['Avg Loss'].replace(to_replace=0, value=0.0001)
    
    # 초기 RSI 값 계산
    dataframe['RSI'] = 100 - (100 / (1 + (dataframe['Avg Gain'] / dataframe['Avg Loss'] + 1e-10)))
    
    # # 초기 기간 이후의 데이터를 선택 (NaN 제거)
    # dataframe = dataframe.iloc[period:]
    # #return dataframe['RSI']
    
    # 시각화
    plt.figure(figsize=(10, 5))
    plt.plot(dataframe.index, dataframe['RSI'], label='RSI')
    plt.axhline(y=70, color='r', linestyle='--', label='Overbought (70)')
    plt.axhline(y=30, color='g', linestyle='--', label='Oversold (30)')
    plt.title('Relative Strength Index (RSI)')
    plt.xlabel('Date')
    plt.ylabel('RSI Value')
    plt.legend()
    return plt.show()
